<a href="https://colab.research.google.com/github/littlexflower/ex9/blob/master/Spotify_Song_Popularity_Update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import glob, requests, time, os

import pandas as pd 
pd.options.mode.chained_assignment = None

from pandas.io.json import json_normalize 

from datetime import date

from google.cloud import bigquery

# enable progress bar when running loops
from tqdm.notebook import tqdm
# enable progress bar when running apply in pandas 
# (use progress_apply instead of apply)
tqdm.pandas()

# disable warnings when running code
import warnings
warnings.filterwarnings("ignore")

### Set essential variables ###
project_path = '/content/drive/MyDrive/Spotify Song Popularity Update'
spotify_popularity_value_retrieved_date = date.today().strftime('%Y-%m-%d')

# Essential variables for BigQuery env
credential_path = '/content/drive/MyDrive/sac_translation.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path
client = bigquery.Client.from_service_account_json(credential_path)
msk_dataset_id = 'music-solutions-data.msk_scheduled.musicdb-pro-max'

### Funtions ###
# Define a function to load a query result into a DataFrame
def get_bq_table(query):
  query_job = client.query(query)
  query_job.result()
  df = query_job.to_dataframe()
  return df

# Define a function to search for a song and return its popularity based on its ISRC on Spotify
def search_a_song_by_isrc(isrc):
    '''
    This function returns a dictionary, including 
    song ID, song name, popularity, 
    album ID, album name, and album release date.
    If there are more than one song returned, choose song with the latest album release date.
    '''
    # this is funkylab account
    client_id = 'c88d0145904d4d3990eda99c27d0ed7e'
    client_secret = 'bfdafee94fb04301a23c2d7c87058d78'
    auth_url = 'https://accounts.spotify.com/api/token'
    # POST
    auth_response = requests.post(auth_url, {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
    })

    # convert the response to JSON
    auth_response_data = auth_response.json()

    # save the access token
    access_token = auth_response_data['access_token']

    headers = {
        'Authorization': f'Bearer {access_token}'
    }

    url = f'https://api.spotify.com/v1/search?type=track&q=isrc:{isrc}'

    responses = requests.get(url, headers=headers)
    result = responses.json()['tracks']['items']
    return result

In [ ]:
targeted_msk_songs = get_bq_table(
    f"""
    WITH 
    song_params_extracted_1 AS (
        SELECT
            app_id
            , song_id
            , JSON_EXTRACT_ARRAY(song_params) song_params_extracted
        FROM `{msk_dataset_id}_app_song`
        WHERE 
          app_id IN ('b14f4f52-7878-4d68-ba01-0e134830728c', '5e70975f-f91f-465a-b7e4-c16981135ea4')
          AND created_at <= '2021-11-04'
    )

    , flatten AS (
        SELECT 
            app_id
            , song_id
            , song_params_extracted 
        FROM song_params_extracted_1, UNNEST(song_params_extracted) song_params_extracted
    )

    , song_params_extracted_2 AS (
        SELECT
            app_id
            , song_id
            , JSON_VALUE(song_params_extracted, '$.key') key
            , JSON_VALUE(song_params_extracted, '$.value') value
        FROM flatten
    )

SELECT 
    app_id msk_app_id
    , song_params_extracted_2.song_id msk_song_id
    , song_name.name msk_song_name
    , isrc
    , value msk_popularity_value
    , '2021-11-04' msk_popularity_value_retrieve_date
FROM song_params_extracted_2
LEFT JOIN `{msk_dataset_id}_song` msk_song 
ON song_params_extracted_2.song_id = msk_song.id
LEFT JOIN `{msk_dataset_id}_song_name` song_name
USING (song_id)
WHERE song_params_extracted_2.key = 'popularity'
    """
)

In [ ]:
total_part = int(len(targeted_msk_songs)/10000) + 1

for part_num in range(0, total_part):
  spotify_search_result = pd.DataFrame()
  start_index = part_num*10000
  end_index = (part_num+1)*10000
  partial_msk_songs = targeted_msk_songs[start_index:end_index]
  for i, row in tqdm(
      partial_msk_songs.iterrows(),
      total = partial_msk_songs.shape[0]
  ):
      isrc = partial_msk_songs.at[i, 'isrc']

      while True:
          try:
              json_normalized = json_normalize(search_a_song_by_isrc(isrc))
              spotify_search_result = spotify_search_result.append(json_normalized)
              time.sleep(0.04)
              break
          except Exception:
              continue

    spotify_search_result.to_csv(f'{project_path}/spotify_search_result_part_{part_num}')

In [ ]:
spotify_search_result = pd.DataFrame()

for file in glob.glob(f'{project_path}/spotify_search_result_part_*'):
  df = pd.read_csv(file)
  spotify_search_result = spotify_search_result.append(df)

spotify_search_result = spotify_search_result[
                                              [
                                               'id',
                                               'name',
                                               'popularity',
                                               'album.release_date',
                                               'album.release_date_precision',
                                               'external_ids.isrc'
                                              ]
]

spotify_search_result['spotify_popularity_value_retrieved_date'] = spotify_popularity_value_retrieved_date

spotify_search_result = spotify_search_result.sort_values('popularity', ascending=False).groupby(['external_ids.isrc']).first().reset_index()

spotify_search_result.columns = [
                                 'isrc',
                                 'spotify_track_id',
                                 'spotify_track_name',
                                 'spotify_popularity_value',
                                 'album_release_date',
                                 'album_release_date_precision',
                                 'spotify_popularity_value_retrieved_date'
]

In [ ]:
popularity_update = targeted_msk_songs.merge(
    spotify_search_result,
    on='isrc'
)

popularity_update.to_csv(f'{project_path}/song_popularity_update_{spotify_popularity_value_retrieved_date}.csv', index=False)

In [2]:
popularity_update = pd.read_csv(f'{project_path}/song_popularity_update_2021-11-05.csv')

In [5]:
popularity_update.head(1000).drop(columns='Unnamed: 0').to_csv(f'{project_path}/song_popularity_update_sample_{spotify_popularity_value_retrieved_date}.csv', index=False)